In [13]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
import tensorflow as tf

## Load data

In [15]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [16]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [17]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [18]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [19]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [20]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [21]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [22]:
seq_len = 128
n_channels = 9

In [23]:
har_X_train = har_X_train.reshape(-1, seq_len * n_channels)
har_X_test = har_X_test.reshape(-1, seq_len * n_channels)

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train,test_size=0.3, random_state=42)

In [45]:
X_train.shape, y_train.shape

((5146, 1152), (5146,))

In [46]:
X_valid.shape

(2206, 1152)

# Build DNN model

## Hyperparameters

In [47]:
batch_size = 600
learning_rate = 0.001
n_epochs = 200

# outputs
n_classes = 6

In [27]:
n_batches = X_train.shape[0] // batch_size

## the model

In [28]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len * n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")


In [29]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 300, activation=tf.nn.relu, name="dense1")
    dense2 = tf.layers.dense(dense1, 100, activation=tf.nn.relu, name="dense2")

    logits = tf.layers.dense(dense2, n_classes)    

In [30]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [31]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_dnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.591115
Epoch:  10 Valid accuracy:  0.93155
Epoch:  20 Valid accuracy:  0.936083
Epoch:  30 Valid accuracy:  0.93019
Epoch:  40 Valid accuracy:  0.933363
Epoch:  50 Valid accuracy:  0.940616
Epoch:  60 Valid accuracy:  0.930644
Epoch:  70 Valid accuracy:  0.93427
Epoch:  80 Valid accuracy:  0.93291
Epoch:  90 Valid accuracy:  0.940617
Epoch:  100 Valid accuracy:  0.951496
Epoch:  110 Valid accuracy:  0.94515
Epoch:  120 Valid accuracy:  0.952403
Epoch:  130 Valid accuracy:  0.94651
Epoch:  140 Valid accuracy:  0.94651
Epoch:  150 Valid accuracy:  0.94107
Epoch:  160 Valid accuracy:  0.943336
Epoch:  170 Valid accuracy:  0.941523
Epoch:  180 Valid accuracy:  0.939257
Epoch:  190 Valid accuracy:  0.940617


In [32]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_dnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_dnn_model.ckpt
Valid accuracy:  0.93291
Test Accuracy:  0.858161


## Add Batch Norm to the model

In tensorflow we use `tf.layers.batch_normalization` for batch normalization
  - first apply the function to each dense layer, including logits layer
  - second need training the `tf.GraphKeys.UPDATE_OPS` after optimizer

fine tuning the model:
  1. training much slowly after batch norm added
  2. use `elu` to replace `relu`, but slow and worse accuracy
  3. use `batch_size` 600, but got worse accuracy

In [49]:
dropout_rate = 0.5

In [50]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len * n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [51]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 80, name="dense1")
    bn1 = tf.layers.batch_normalization(dense1, training=training)
    bn1_act = tf.nn.relu(bn1)
    drop1 = tf.layers.dropout(bn1_act, dropout_rate, training=training)
    
    dense2 = tf.layers.dense(drop1, 80, name="dense2")
    bn2 = tf.layers.batch_normalization(dense2, training=training)
    bn2_act = tf.nn.relu(bn2)
    drop2 = tf.layers.dropout(bn2_act, dropout_rate, training=training)
    
    #dense3 = tf.layers.dense(drop2, 50, name="dense3")
    #bn3 = tf.layers.batch_normalization(dense3, training=training)
    #bn3_act = tf.nn.relu(bn3)
    #drop3 = tf.layers.dropout(bn3_act, dropout_rate, training=training)
    
    logits_before_bn = tf.layers.dense(drop2, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)


In [52]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):   
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) 

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [53]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training:True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_dnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.444243
Epoch:  10 Valid accuracy:  0.713055
Epoch:  20 Valid accuracy:  0.893472
Epoch:  30 Valid accuracy:  0.922937
Epoch:  40 Valid accuracy:  0.926564
Epoch:  50 Valid accuracy:  0.929284
Epoch:  60 Valid accuracy:  0.931097
Epoch:  70 Valid accuracy:  0.93563
Epoch:  80 Valid accuracy:  0.94651
Epoch:  90 Valid accuracy:  0.94107
Epoch:  100 Valid accuracy:  0.939257
Epoch:  110 Valid accuracy:  0.950136
Epoch:  120 Valid accuracy:  0.941976
Epoch:  130 Valid accuracy:  0.949229
Epoch:  140 Valid accuracy:  0.946963
Epoch:  150 Valid accuracy:  0.943336
Epoch:  160 Valid accuracy:  0.93699
Epoch:  170 Valid accuracy:  0.946509
Epoch:  180 Valid accuracy:  0.94651
Epoch:  190 Valid accuracy:  0.948776


In [54]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_dnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_dnn_model.ckpt
Valid accuracy:  0.93971
Test Accuracy:  0.896844
